In [1]:
import os
import sqlite3
from pathlib import Path
import pandas as pd
import shutil

#gather data from a specific csv file and return as a pandas df
BASE = Path.cwd()
DATA = BASE / "training_data"
#Make sure training_data folder exists
DATA.mkdir(parents= True, exist_ok= True)

In [ ]:
#TO USE THIS API you must have a .kaggle folder in your 'C:\NAME' directory -> then paste the kaggle.json authenticator
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

print(api.dataset_list_files('wordsforthewise/lending-club').files)

api.dataset_download_files('wordsforthewise/lending-club', path= DATA, unzip=True)

In [ ]:
def retrieve_training_csv(): 
    """ Function that returns all the csv files in the training data folder as a dataframe object """ 
    csv_list = list(DATA.glob("**/*.csv")) 
    return_list = [] 
    print(csv_list)

    #traverse each item inthe data path, and if a file ending in .csv is found, turn it into a df and append to return list 
    for item in csv_list: 
        if os.path.isfile(item): 
            print(f"{item} is a file") 
            return_list.append(pd.read_csv(item)) 
        else: print(f"{item} is folder or dir")

    if return_list: 
        return return_list 
    else: 
        print(f"No csv files found in {DATA}") 
        return []

def get_dir_size(path): 
    """ Get directory size in MBs """ 
    total = 0 
    for dirpath, dirs, files in os.walk(path):
        for f in files:
            file_path = os.path.join(dirpath, f)
            
            try:
                total += os.path.getsize(file_path) 
                print(f"{total/1000000} MB")
            except: 
                continue 
        return total/1000000
    
def delete_large_files(): 
    """ Function that deletes the large files downloaded from kaggle to save space 
    Deletes files to prevent any storage errors when pushing code to github """

    for paths in DATA.glob("**/*"): 
        if os.path.isdir(paths): 
            size = get_dir_size(paths) 
            print(f"size of {paths} is {size}")

        if size > 100: 
            try: 
                print(f"deleting {paths}") 
                shutil.rmtree(paths) 
            except Exception as e:
                print(f"Error: {e} when attempting to delete {paths}")

df_list = retrieve_training_csv()
delete_large_files()

In [ ]:
#create different pointers to the csvs in the list for easier access
accepted_loans = df_list[0]
rejected_loans = df_list[1]
loans_paid_info = df_list[2]